In [1]:
import os
import pickle
import pandas as pd
from tqdm import tqdm

import warnings
warnings.filterwarnings('ignore')

DATA_PATH = "../data"
DATA_FILENAME = "data.csv"
data_file = os.path.join(DATA_PATH, DATA_FILENAME)

In [2]:
iter_csv = pd.read_csv(data_file, encoding="utf-16", sep='\t', iterator=True, chunksize=1000000)
df = pd.concat([chunk[(chunk['Закупки по'] == '44-ФЗ') 
                & (chunk['Этап закупки']=='Определение поставщика завершено')
                & (chunk["Дата обновления"].str.split('.', expand=True).iloc[:, 2].astype("int").isin([2019, 2020, 2021]))] 
                for chunk in tqdm(iter_csv)])

11it [04:12, 22.98s/it]


In [3]:
df['purch'] = df['Реестровый номер закупки'].apply(lambda x: x[1:])
df["Дата обновления"] = pd.to_datetime(df["Дата обновления"], format="%d.%m.%Y")
df["Год обновления"] = df["Дата обновления"].apply(lambda date: date.year)

In [4]:
purch_to_year = df[["purch", "Год обновления"]].reset_index(drop=True).set_index("purch").to_dict()
purch_to_sum = df[["purch", "Начальная (максимальная) цена контракта"]].reset_index(drop=True).set_index("purch").to_dict()

In [5]:
df1 = pd.read_csv(os.path.join(DATA_PATH, "async_info_1.csv"), dtype={"purch": str, "inn": str})
df2 = pd.read_csv(os.path.join(DATA_PATH, "async_info_2.csv"), dtype={"purch": str, "inn": str})
df3 = pd.read_csv(os.path.join(DATA_PATH, "async_info_3.csv"), dtype={"purch": str, "inn": str})

In [6]:
info_df = pd.concat((df1, df2, df3))

In [7]:
info_df["year"] = info_df["purch"].apply(lambda p: purch_to_year["Год обновления"][p])
info_df["amount"] = info_df["purch"].apply(lambda p: purch_to_sum["Начальная (максимальная) цена контракта"][p])

In [8]:
info_df = info_df[(info_df.inn.notna()) | (info_df.name.notna()) | (info_df.phone.notna())]

In [16]:
count_df = info_df.pivot_table("purch", index="inn", columns=["year"], aggfunc='count')
count_df = count_df.fillna(0)
count_df

year,2019,2020,2021
inn,,,
0105004524,0.0,1.0,1.0
0105071351,1.0,1.0,1.0
021103095256,0.0,2.0,3.0
021700659227,0.0,1.0,1.0
022003004218,0.0,1.0,1.0
...,...,...,...
9725033183,0.0,1.0,3.0
9728014037,0.0,2.0,2.0
9731067536,0.0,1.0,7.0


In [18]:
sum_df = info_df.pivot_table("amount", index="inn", columns=["year"], aggfunc='sum')
sum_df = sum_df.fillna(0)
sum_df

year,2019,2020,2021
inn,,,
0105004524,0.00,35268.75,3353920.00
0105071351,378393.25,2902500.00,5573312.40
021103095256,0.00,2135951.00,4845027.00
021700659227,0.00,250000.00,1690880.00
022003004218,0.00,500000.00,500000.00
...,...,...,...
9725033183,0.00,2997164.54,7904310.58
9728014037,0.00,18762995.63,10492818.84
9731067536,0.00,5565914.00,21615726.99


In [44]:
#count_df = count_df.rename({2019: "Кол-во 2019", 2020: "Кол-во 2020", 2021: "Кол-во 2021"}, axis=1)
sum_df = sum_df.rename({2019: "Сумма 2019", 2020: "Сумма 2020", 2021: "Сумма 2021"}, axis=1)

In [10]:
cond2 = count_df[(count_df[2020] != 0) & ((count_df[2021] / count_df[2020]) >= 0.75)].index

In [11]:
mean_amounts = info_df[info_df.year != 2019].groupby("inn").amount.mean()
mean_amounts = mean_amounts[(mean_amounts >= 500000) & (mean_amounts <= 30000000)]

In [12]:
cond1 = mean_amounts.index

In [56]:
mean_amounts["773271002265"]

897023.6533333333

In [13]:
info_df = info_df[(info_df.inn.isin(cond1)) & (info_df.inn.isin(cond2))]

In [20]:
info_df = info_df.drop_duplicates(subset=["inn"])

In [25]:
info_df.drop("amount", axis=1, inplace=True)

In [45]:
merged_info_df = info_df.merge(count_df, how="inner", on="inn").merge(sum_df, how="inner", on="inn")

In [52]:
merged_info_df

,purch,inn,name,phone,address,email,Кол-во 2019,Кол-во 2020,Кол-во 2021,Сумма 2019,Сумма 2020,Сумма 2021
0,0373200032219000110,773271002265,ИП Ващенко Виталий АлександровичИндивидуальный...,+7 (915) 129-45-72,"г. Москва, ул. Лукинская, дом 1, кв. 92",vashenkoip@yandex.ru,3.0,1.0,2.0,19878965.48,735912.29,1955158.67
1,0149200002319000054,7813600141,Общество с ограниченной ответственностью «Соци...,+7(812)3808032,"197110, Российская Федерация, г. Санкт-Петербу...",support@social-soft.ru,4.0,2.0,3.0,4669763.00,5008347.00,7146851.00
2,0306200011519000029,6147011500,ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ТОРГО...,8-6365-70833,"347810, ОБЛ РОСТОВСКАЯ 61, Г КАМЕНСК-ШАХТИНСКИ...",arix5@rambler.ru,7.0,5.0,6.0,8828845.40,1839990.37,18024308.34
3,0373200041519000080,7715549708,"АНО ""Катаржина""",+7 (499) 205-90-10,"127106 город Москва, улица Комдива Орлова, дом 4",info@katarzyna.ru,1.0,1.0,1.0,3938383.50,1508237.08,349000.01
4,0356500002619000005,5904122072,Федеральное бюджетное учреждение здравоохранен...,7-342-2393409,"614016, КРАЙ ПЕРМСКИЙ, ГОРОД ПЕРМЬ, УЛИЦА КУЙБ...",dogcgepo@bk.ru,3.0,3.0,3.0,959200.34,2195832.40,2544504.92
...,...,...,...,...,...,...,...,...,...,...,...,...
1475,0166100001121000005,7107503225,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""КОПИ...",7-4872-367591,"300012, ОБЛ ТУЛЬСКАЯ 71, Г ТУЛА, ПР-КТ ЛЕНИНА,...",kmservis71@mail.ru,0.0,1.0,1.0,0.00,750000.00,1050000.00
1476,0873100007321000019,9718118661,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""ЭКОВ...",7-967-0498708,"107497, Г МОСКВА 77, ПРОЕЗД 2-Й ИРТЫШСКИЙ, ДОМ...",ecovelikan@gmail.ru,0.0,1.0,1.0,0.00,15120753.90,14191200.00
1477,0122200002521003697,2723003367,ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ПРЕДП...,79142035972,"680020, КРАЙ ХАБАРОВСКИЙ 27, Г ХАБАРОВСК, ПЕР ...",izotop@bk.ru,0.0,1.0,1.0,0.00,5931999.96,88537.08
1478,0190200000320012257,781418325084,КУРИЛЕНКО АНАТОЛИЙ ВАЛЕРЬЕВИЧ -Индивидуальный ...,7-812-7035355,"Г САНКТ-ПЕТЕРБУРГ78,",medkontakt@medkontakt.spb.ru,0.0,1.0,1.0,0.00,1024920.00,1210000.00


In [49]:
merged_info_df.to_excel(os.path.join(DATA_PATH, "info.xlsx"), index=False)